<a href="https://colab.research.google.com/github/UmutSonmezz/ComputerVision/blob/main/Image_Classification_Tensorflow_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall keras
!pip install keras==2.15.0
!pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
IMAGE_SIZE=[224,224]
train_path='/content/drive/MyDrive/Computer_Vision/train'
valid_path='/content/drive/MyDrive/Computer_Vision/validation'
test_path='/content/drive/MyDrive/Computer_Vision/test'

In [ ]:
vgg16=VGG16(input_shape = IMAGE_SIZE +[3], weights="imagenet",include_top=False)

In [ ]:
for layer in vgg16.layers:
  layer.trainable=False

In [ ]:
folder=glob('/content/drive/MyDrive/Computer_Vision/train/*')

In [ ]:
folder

In [ ]:
model = Sequential()
model.add(vgg16)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense (2, activation='softmax'))

In [ ]:
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

In [ ]:
#Reproducing data for education
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
validation_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
#Uploading Tutorial Images
training_set=train_datagen.flow_from_directory(
"/content/drive/MyDrive/Computer_Vision/train",
  target_size=(224,224),
  batch_size=32,
  class_mode="categorical")


In [ ]:
#Uploading Validation Images
validation_set = validation_datagen.flow_from_directory("/content/drive/MyDrive/Computer_Vision/validation",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical")

In [ ]:
#test_set = validation_datagen.flow_from_directory("/content/drive/MyDrive/Computer_Vision/test",
    #target_size=(224, 224),
    #class_mode=None)

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',
                               patience=3,
                               restore_best_weights=True)

In [ ]:
history = model.fit(
    training_set,
    validation_data=validation_set,
    epochs=10,
    steps_per_epoch=training_set.n // training_set.batch_size,
    validation_steps=validation_set.n // validation_set.batch_size
)

In [ ]:
history.history

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'],label="train loss")
plt.plot(history.history['val_loss'],label="validation loss")
plt.xlabel("epoch")
plt.ylabel("loss")
plt.legend()
plt.show()
plt.savefig("loss graph")

In [ ]:
#Save Model
model.save("model_vgg16.h5", save_format='h5')

In [ ]:
!pip install --upgrade h5py

In [ ]:
#Load Model
from tensorflow.keras.models import load_model
model=load_model("/content/drive/MyDrive/Computer_Vision/model_vgg16.h5")

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
def load_test_images(directory, target_size=(224, 224)):
    images = []
    filenames = []
    for filename in os.listdir(directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=target_size)
            img_array = img_to_array(img)
            images.append(img_array)
            filenames.append(filename)
    return np.array(images), filenames

test_dir = "/content/drive/MyDrive/Computer_Vision/test"
test_images, filenames = load_test_images(test_dir)
test_images=test_images / 255.0


In [ ]:
print(f"Yüklenen test resmi sayısı: {len(test_images)}")
print(f"Test resimlerinin boyutu: {test_images.shape}")

In [ ]:
y_pred=model.predict(test_images)

In [ ]:
y_pred

In [ ]:
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
y_pred

In [ ]:
image_name = "/content/drive/MyDrive/Computer_Vision/dog.10455.jpg"

In [ ]:
img =image.load_img("/content/drive/MyDrive/Computer_Vision/dog.10455.jpg" , target_size=(224, 224))

In [ ]:
x=image.img_to_array(img)
x

In [ ]:
x.shape

In [ ]:
x=x/255

In [ ]:
from keras.applications.vgg16 import preprocess_input
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

**np.expand_dims(x, axis=0)**: Adds a batch dimension to the image.

**preprocess_input(x)**: Processes (normalizes) the image data to be compatible with the VGG16 model.

**img_data.shape**: Provides the shape of the processed data, which will typically be (1, 224, 224, 3), indicating that the image is ready to be processed by the model.

In [ ]:
model.predict(img_data)

In [ ]:
result=np.argmax(model.predict(img_data), axis=1)

In [ ]:
result